### Data Transformation

In [1]:
1+1

2

In [6]:
import os
%pwd

'/media/bhuvanesh/OS/Users/bhuva/Documents/Learnings/End-to-End MLOps Bootcamp/textsummarizer'

In [7]:
os.chdir("TextSummarizer")

In [8]:
%pwd

'/media/bhuvanesh/OS/Users/bhuva/Documents/Learnings/End-to-End MLOps Bootcamp/textsummarizer/TextSummarizer'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [11]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml,create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.paramss=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config



In [13]:
import os
from src.TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

[2025-06-05 16:29:55,665: INFO: config: PyTorch version 2.7.0 available.]


### Data Transformation Component


In [14]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


    

In [15]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()


[2025-06-05 16:30:08,450: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-05 16:30:08,455: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-05 16:30:08,457: INFO: common: created directory at: artifacts]
[2025-06-05 16:30:08,460: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/media/bhuvanesh/OS/Users/bhuva/Documents/Learnings/End-to-End MLOps Bootcamp/textsummarizer/TextSummarizer/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]